# Healthcare Infrastructure

In this notebook, we will perform a **damage and risk assessment** for healthcare infrastructure, specifically focusing on hospitals and clinics. The assessment is based on combining hazard data (e.g., flood depths) with vulnerability curves to estimate the potential damage to infrastructure.

We will follow the steps outlined below to conduct the assessment:

1. **Loading the necessary packages:**  
   We will import the Python libraries required for data handling, analysis, and visualization.

2. **Loading the data:**  
   The infrastructure data (e.g., hospitals) and hazard data (e.g., flood depths) will be loaded into the notebook.

3. **Preparing the data:**  
   The infrastructure and hazard data will be processed and data gaps can be filled, if required.

4. **Performing the damage assessment:**  
   We will apply vulnerability curves to estimate the potential damage based on the intensity of the hazard.

5. **Visualizing the results:**  
   Finally, we will visualize the estimated damage using graphs and maps to illustrate the impact on healthcare infrastructure.

## 1. Loading the Necessary Packages
To perform the assessment, we are going to make use of several [python packages](https://docs.python.org/3/tutorial/modules.html#packages).

In case you run this in Google Colab, you will have to install the packages below **(remove the hashtag in front of them)**.

In [ ]:
#!pip install damagescanner==0.9b12
#!pip install contextily
#!pip install exactextract
#!pip install lonboard

In this step, we will import all the required Python libraries for data manipulation, spatial analysis, and visualization.

In [ ]:
import warnings
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

import matplotlib.pyplot as plt
import contextily as cx

import damagescanner.download as download
from damagescanner.core import DamageScanner
from lonboard import viz

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning) # exactextract gives a warning that is invalid

### Specify the country of interest

Before we continue, we should specify the country for which we want to assess the damage. We use the ISO3 code for the country to download the OpenStreetMap data.

In [ ]:
country_full_name = 'Jamaica'
country_iso3 = 'JAM'

## 2. Loading the Data
In this step, we will load four key datasets:

1. **Infrastructure data:**  
   This dataset contains information on the location and type of transportation infrastructure (e.g., roads). Each asset may have attributes such as type, length, and replacement cost.

2. **Hazard data:**  
   This dataset includes information on the hazard affecting the infrastructure (e.g., flood depth at various locations).

3. **Vulnerability curves:**  
   Vulnerability curves define how the infrastructure's damage increases with the intensity of the hazard. For example, flood depth-damage curves specify how much damage occurs for different flood depths.

4. **Maximum damage values:**  
   This dataset contains the maximum possible damage (in monetary terms) that can be incurred by individual infrastructure assets. This is crucial for calculating total damage based on the vulnerability curves.

### Infrastructure Data

We will perform this example analysis for Jamaica. To start the analysis, we first download the OpenStreetMap data from GeoFabrik. 

In [ ]:
infrastructure_path = download.get_country_geofabrik(country_iso3)

Now we load the data and read only the healthcare data.

In [ ]:
%%time
features = read_osm_data(infrastructure_path,asset_type='healthcare')

In [ ]:
sub_types = features.object_type.unique()

And we can explore our data interactively 

In [ ]:
viz(features)

### Hazard Data
For this example, we make use of the flood data provided by [CDRI](https://giri.unepgrid.ch/map?list=explore).

We use a 1/100 flood map to showcase the approach.

In [ ]:
hazard_map = xr.open_dataset("https://hazards-data.unepgrid.ch/global_pc_h100glob.tif", engine="rasterio")

In [ ]:
hazard_map

### Maximum damages
One of the most difficult parts of the assessment is to define the maximum reconstruction cost of particular assets. Here we provide a baseline set of values, but these should be updated through national consultations.

Locations of healthcare facilities are (somewhat randomly) geotagged as either points or polygons. This matters quite a lot for the maximum damages. For polygons, we would use damage per square meter, whereas for points, we would estimate the damage to the entire asset at once. Here we take the approach of converting the points to polygons, and there define our maximum damages in dollar per square meter. 

In [ ]:
maxdam_dict = {'clinic' : 1000, 
               'dentist' : 1000, 
               'laboratory' : 1000, 
               'doctors' : 1000, 
               'pharmacy' : 1000,
               'hospital' : 1000,
               'alternative' : 1000 }

To be used in our damage assessment, we convert this to a Pandas DataFrame

In [ ]:
maxdam = pd.DataFrame.from_dict(maxdam_dict,orient='index').reset_index()
maxdam.columns = ['object_type','damage']

### Vulnerability data
Similarly to the maximum damages, specifying the vulnerability curves is complex. We generally have limited information about the quality of the assets, its level of deteriation and other asset-level characteristics.

In [ ]:
vulnerability_path = "https://zenodo.org/records/10203846/files/Table_D2_Multi-Hazard_Fragility_and_Vulnerability_Curves_V1.0.0.xlsx?download=1"
vul_df = pd.read_excel(vulnerability_path,sheet_name='F_Vuln_Depth')

And let's have a look at all the available options

In [ ]:
with pd.option_context('display.max_rows', None):
  display(vul_df.iloc[:2,:].T)

And select a curve to use for each different subtype we are analysing.

In [ ]:
sub_types

In [ ]:
selected_curves = dict(zip(sub_types,['F21.1','F21.1','F21.2','F21.1','F21.2','F21.3','F21.1']))

In [ ]:
damage_curves = vul_df[['ID number']+list(selected_curves.values())]
damage_curves = damage_curves.iloc[4:125,:]
damage_curves.set_index('ID number',inplace=True)
damage_curves.index = damage_curves.index.rename('Depth')  
damage_curves = damage_curves.astype(np.float32)
damage_curves.columns = sub_types

Make sure we set the index of the damage curves (the inundation depth) in the same metric as the hazard data (e.g. meters or centimeters).

In [ ]:
damage_curves.index = damage_curves.index*100

### Ancilliary data for processing

In [ ]:
world = gpd.read_file("https://github.com/nvkelso/natural-earth-vector/raw/master/10m_cultural/ne_10m_admin_0_countries.shp")
world_plot = world.to_crs(3857)

In [ ]:
admin1 = gpd.read_file("https://github.com/nvkelso/natural-earth-vector/raw/master/10m_cultural/ne_10m_admin_1_states_provinces.shp")

## 3. Preparing the Data

Clip the hazard data to the country of interest.

In [ ]:
country_bounds = world.loc[world.ADM0_ISO == country_iso3].bounds
country_geom = world.loc[world.ADM0_ISO == country_iso3].geometry

In [ ]:
hazard_country = hazard_map.rio.clip_box(minx=country_bounds.minx.values[0],
                     miny=country_bounds.miny.values[0],
                     maxx=country_bounds.maxx.values[0],
                     maxy=country_bounds.maxy.values[0]
                    )

Convert the point data to polygons. We first try to estimate an average size of each building based on each subtype.

To do so, let's first grab the polygon data and estimate their size:

In [ ]:
polygon_features = features.loc[features.geometry.geom_type.isin(['Polygon','MultiPolygon'])].to_crs(3857)
polygon_features['square_m2'] = polygon_features.area

In [ ]:
square_m2_object_type = polygon_features[['object_type','square_m2']].groupby('object_type').median()

In [ ]:
square_m2_object_type

It seems we are missing some features. Let's check which one:

In [ ]:
list(set(sub_types)-set(square_m2_object_type.index))

Now we can add this to our list. Let's use pharmacy as the square_m2 size.

In [ ]:
square_m2_object_type.loc['laboratory'] = square_m2_object_type.loc['pharmacy','square_m2']

And now we can use this to create polygons, using this square meters as the average size for each point.

In [ ]:
features.loc[features.geometry.geom_type == 'Point','geometry'] = features.loc[features.geometry.geom_type == 'Point'].to_crs(3857).apply(lambda x: 
                                                                    shapely.buffer(x.geometry, 
                                                                    distance=np.sqrt(square_m2_object_type.loc[x.object_type,'square_m2']),
                                                                    cap_style='square'),axis=1).set_crs(3857).to_crs(4326)

## 5. Performing the Damage Assessment
We will use the DamageScanner approach. This is a fully optimised damage calculation method, that can capture a wide range of inputs to perform a damage assessment.

In [ ]:
%%time
damage_results = DamageScanner(hazard_country, features, damage_curves, maxdam).calculate()

## 5. Save the Results
For further analysis, we can save the results in their full detail, or save summary estimates per subnational unit 

In [ ]:
hazard = 'river_flood'
return_period = '1_100'
damage_results.csv(f'Healthcare_Damage_{country_full_name}_{hazard}_{return_period}.csv')

In [ ]:
admin1_country = admin1.loc[admin1.sov_a3 == country_iso3]

In [ ]:
damage_results = damage_results.sjoin(admin1_country[['adm1_code','name','geometry']])

In [ ]:
admin1_damage = admin1_country.merge(damage_results[['name_right','damage']].groupby('name_right').sum(),
                                     left_on='name',
                                     right_on='name_right',
                                     how='outer')[['name','adm1_code','geometry','damage']]

In [ ]:
admin1_damage.to_csv(f'Admin1_Road_Damage_{country_full_name}_{hazard}_{return_period}.csv')

## 6. Visualizing the Results
The results of the damage assessment can be visualized using charts and maps. This will provide a clear representation of which infrastructure is most affected by the hazard and the expected damage levels.

And create a distribution of the damages.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(7, 3))

sns.histplot(data=damage_results,x='damage',ax=ax)

Plot location of most damaged healthcare facilities

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

subset = damage_results.to_crs(3857).sort_values('damage',ascending=False).head(20)
subset.geometry = subset.centroid
subset.plot(ax=ax,column='object_type',markersize=10,legend=True)
world_plot.loc[world.SOV_A3 == country_iso3].plot(ax=ax,facecolor="none",edgecolor='black')
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron,alpha=0.5)
ax.set_axis_off()

In [ ]:
viz(damage_results)